In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
from dotenv import load_dotenv
load_dotenv()

import os
gemini_api_key=os.getenv("GEMINI_API_KEY")
gemini_model_name=os.getenv("GEMINI_MODEL_NAME")
print(gemini_model_name)


gemini/gemini-2.5-flash


In [20]:
from crewai import LLM

llm = LLM(
    model=gemini_model_name,
    api_key=gemini_api_key,
    temperature=0.7
)
# response = llm.call(
#     "Analyze the following messages and return the name, age, and breed. "
#     "Meet Kona! She is 3 years old and is a black german shepherd."
# )
# print(response)

Creating a planner

In [21]:
from crewai import Agent, Task, Crew

In [22]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

Agent : Writer

In [23]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

Agent:Editor

In [24]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

Task:Plan

In [25]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

Task: write

In [26]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

Task: Edit

In [27]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

creating crew:

In [28]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

running crew

In [29]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0ad1a775-2310-40dc-a6ce-37e0d0144832                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.            │
│  2. Identify the target audience, considering their interests and pain points.                                  │
│  3. Develop a detailed content outline including an introduction, key points, and a call to action.             │
│  4. Include SEO keywords and relevant data or sources.                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **AI Blog Article Content Plan Document**                                                                      │
│                                                                                                                 │
│  **Article Title Options:**                                                                                     │
│                                                                                                                 │
│  *   The AI Revolution: Understanding the Past, Navigating the Present, and Shaping the Future                  │
│  *   Beyond the Hype: A Comprehensive Guide to Artificial Intelligence in 2024                                  │
│  *   Unlocking AI: Trends, Impact, and What You Need to Know Now                                                │
│  *   Artificial Intelligence Explained: Your Essential Guide to the AI Era                                      │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Target Audience Analysis:**                                                                                  │
│                                                                                                                 │
│  *   **Demographics:** Professionals (managers, developers, marketers, educators), small business owners, tech  │
│  enthusiasts, students, and general readers interested in technology's impact on society. Age range typically   │
│  25-60, with a moderate to high level of education.                                                             │
│  *   **Interests:**                                                                                             │
│      *   Understanding complex AI concepts in an accessible way.                                                │
│      *   How AI can improve productivity, efficiency, and innovation in their work or business.                 │
│      *   Staying updated on the latest AI trends, tools, and breakthroughs.                                     │
│      *   Career opportunities and skill development in the AI era.                                              │
│      *   The ethical, social, and economic implications of AI.                                                  │
│      *   Practical applications of AI in everyday life and various industries.                                  │
│  *   **Pain Points:**                                                                                           │
│      *   Feeling overwhelmed or left behind by the rapid pace of AI advancements.                               │
│      *   Difficulty distinguishing between hype and reality in AI news.                                         │
│      *   Concerns about job displacement, data privacy, and algorithmic bias.                                   │
│      *   Lack of clear, reliable, and unbiased information about AI.                                            │
│      *   Uncertainty about how to integrate AI tools into their workflow or business strategy.                  │
│      *   Fear of the unknown regarding AI's future impa

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 74c80f7d-a8c0-4b0e-ab10-63cd7d06153c                                                                     │
│  Agent: Content Planner                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Writer                                                                                          │
│                                                                                                                 │
│  Task: 1. Use the content plan to craft a compelling blog post on Artificial Intelligence.                      │
│  2. Incorporate SEO keywords naturally.                                                                         │
│  3. Sections/Subtitles are properly named in an engaging manner.                                                │
│  4. Ensure the post is structured with an engaging introduction, insightful body, and a summarizing             │
│  conclusion.                                                                                                    │
│  5. Proofread for grammatical errors and alignment with the brand's voice.                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Task: Proofread the given blog post for grammatical errors and alignment with the brand's voice.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Beyond the Hype: A Comprehensive Guide to Artificial Intelligence in 2024                                    │
│                                                                                                                 │
│  In 2024, it's virtually impossible to avoid the topic of Artificial Intelligence (AI). From powering our       │
│  smartphone assistants to revolutionizing scientific research, **AI** is no longer a futuristic concept but a   │
│  tangible force reshaping our daily lives and global industries. The sheer speed of **AI trends** and           │
│  breakthroughs can feel overwhelming, leaving many wondering: what exactly is **Artificial Intelligence**, and  │
│  what does its pervasive presence mean for us?                                                                  │
│                                                                                                                 │
│  This article aims to demystify the complex world of **AI**, offering a clear, engaging, and factually          │
│  accurate overview of its current state, key trends, and transformative impact. My goal is to empower you with  │
│  foundational knowledge, help you navigate the rapidly evolving **AI landscape**, and encourage informed        │
│  decision-making regarding **AI's role** in your personal and professional life. We'll explore everything from  │
│  its basic definitions to its ethical implications, providing a comprehensive understanding of the **AI Era**.  │
│                                                                                                                 │
│  ## What is AI? A Quick Primer                                                                                  │
│                                                                                                                 │
│  At its core, **Artificial Intelligence** refers to the simulation of human intelligence processes by           │
│  machines, especially computer systems. These processes include learning (the acquisition of information and    │
│  rules for using the information), reasoning (using rules to reach approximate or definite conclusions), and    │
│  self-correction. It's far more than just robots; it's the intelligence exhibited by machines, allowing them    │
│  to perceive their environment and take actions that maximize their chance of achieving a goal.                 │
│                                                                                                                 │
│  The broad field of **AI** encompasses several key branches that power most of the advanced **AI tools** we     │
│  see today. **Machine Learning (ML)** enables systems to learn from data without explicit programming,          │
│  utilizing algorithms like supervised, unsupervised, and reinforcement learning. A subset of ML, **Deep         │
│  Learning (DL)**, uses neural networks with multiple layers to learn complex patterns, driving advancements in  │
│  areas like image and speech recognition. Meanwhile, **Natural Language Processing (NLP)** empowers machines    │
│  to understand, interpret, and generate human language, while **Computer Vision (CV)** allows them to "see"     │
│  and interpret visual information. A particularly excit

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 167b9e4c-7387-4943-9e5f-6b1283dafa07                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0ad1a775-2310-40dc-a6ce-37e0d0144832                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Beyond the Hype: A Comprehensive Guide to Artificial Intelligence in 2024                                    │
│                                                                                                                 │
│  In 2024, it's virtually impossible to avoid the topic of Artificial Intelligence (AI). From powering our       │
│  smartphone assistants to revolutionizing scientific research, **AI** is no longer a futuristic concept but a   │
│  tangible force reshaping our daily lives and global industries. The sheer speed of **AI trends** and           │
│  breakthroughs can feel overwhelming, leaving many wondering: what exactly is **Artificial Intelligence**, and  │
│  what does its pervasive presence mean for us?                                                                  │
│                                                                                                                 │
│  This article aims to demystify the complex world of **AI**, offering a clear, engaging, and factually          │
│  accurate overview of its current state, key trends, and transformative impact. My goal is to empower you with  │
│  foundational knowledge, help you navigate the rapidly evolving **AI landscape**, and encourage informed        │
│  decision-making regarding **AI's role** in your personal and professional life. We'll explore everything from  │
│  its basic definitions to its ethical implications, providing a comprehensive understanding of the **AI Era**.  │
│                                                                                                                 │
│  ## What is AI? A Quick Primer                                                                                  │
│                                                                                                                 │
│  At its core, **Artificial Intelligence** refers to the simulation of human intelligence processes by           │
│  machines, especially computer systems. These processes include learning (the acquisition of information and    │
│  rules for using the information), reasoning (using rules to reach approximate or definite conclusions), and    │
│  self-correction. It's far more than just robots; it's the intelligence exhibited by machines, allowing them    │
│  to perceive their environment and take actions that maximize their chance of achieving a goal.                 │
│                                                                                                                 │
│  The broad field of **AI** encompasses several key branches that power most of the advanced **AI tools** we     │
│  see today. **Machine Learning (ML)** enables systems to learn from data without explicit programming,          │
│  utilizing algorithms like supervised, unsupervised, and reinforcement learning. A subset of ML, **Deep         │
│  Learning (DL)**, uses neural networks with multiple layers to learn complex patterns, driving advancements in  │
│  areas like image and speech recognition. Meanwhile, **Natural Language Processing (NLP)** empowers machines    │
│  to understand, interpret, and generate human language

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [46]:
# from IPython.display import Markdown
# Markdown(result.raw)

# from IPython.display import display, Markdown, Latex,
# display(Markdown(result.raw))
# # If you particularly want to display maths, this is more direct:
# display(Latex(result.raw))

In [47]:
from IPython.display import display_markdown
display_markdown(result, raw=True)
